In [1]:
special_questpairs = [83633,242349]

In [2]:
import pandas as pd
import numpy as np
import seaborn as sbn
%matplotlib inline
import pylab as plt
from src import cleaning, model
from src.posfeatures import POS_Tagger

In [3]:
from src import data
reload(data)

<module 'src.data' from 'src/data.pyc'>

In [4]:
train = data.DataContainer('data/train.csv')

In [5]:
train.clean_questions()

In [6]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [7]:
corpus = train.question1.tolist() + train.question2.tolist()
documents = [TaggedDocument(s.split(), [i]) for i, s in enumerate(corpus) if isinstance(s, unicode)]
print documents[:10]



[TaggedDocument(words=[u'What', u'is', u'the', u'step', u'by', u'step', u'guide', u'to', u'invest', u'in', u'share', u'market', u'in', u'india'], tags=[0]), TaggedDocument(words=[u'What', u'is', u'the', u'story', u'of', u'Kohinoor', u'Koh', u'i', u'Noor', u'Diamond'], tags=[1]), TaggedDocument(words=[u'How', u'can', u'I', u'increase', u'the', u'speed', u'of', u'my', u'internet', u'connection', u'while', u'using', u'a', u'VPN'], tags=[2]), TaggedDocument(words=[u'Why', u'am', u'I', u'mentally', u'very', u'lonely', u'How', u'can', u'I', u'solve', u'it'], tags=[3]), TaggedDocument(words=[u'Which', u'one', u'dissolve', u'in', u'water', u'quikly', u'sugar', u'salt', u'methane', u'and', u'carbon', u'di', u'oxide'], tags=[4]), TaggedDocument(words=[u'Astrology', u'I', u'am', u'a', u'Capricorn', u'Sun', u'Cap', u'moon', u'and', u'cap', u'rising', u'what', u'does', u'that', u'say', u'about', u'me'], tags=[5]), TaggedDocument(words=[u'Should', u'I', u'buy', u'tiago'], tags=[6]), TaggedDocument(w

In [ ]:
model = Doc2Vec(size = 300, min_count = 1, iter = 55)
model.build_vocab(documents)
model.train(documents)

In [ ]:

q1vecs = train.question1.apply(lambda x: model.infer_vector(x.split()))
q2vecs = train.question1.apply(lambda x: model.infer_vector(x.split()))

print q1vecs.loc[0]

In [ ]:
train.get_char_features(suffix = '_stops')
train.get_char_features(remove_stopwords = True)
train.get_pos_features()
train.get_ngram_features(n = 1, suffix = '_stops')
train.get_ngram_features(n = 1, remove_stopwords=True)
train.get_ngram_features(n = 2, suffix = '_stops')
train.get_ngram_features(n = 2, remove_stopwords = True)
train.get_fuzzy_features(remove_stopwords = False, suffix = '_stops')
train.get_fuzzy_features(remove_stopwords = True, suffix = '')
train.get_vector_features(vectorizer = 'QuoraQuestions', n_principle_directions = 30, suffix = 'QQ')
train.get_vector_features(vectorizer = 'GoogleNews', n_principle_directions = 30, suffix = 'GG')

In [ ]:
[c for c in train.features.columns]
train.features.shape
#from scipy.spatial import distance
#distance.cosine(np.ones(20), np.arange(20))

In [ ]:
index = 242349
train.question1.loc[index], train.question2.loc[index]


In [ ]:
bst = model.train_xgb(train.features,train.target, num_rounds = 200, eta = 0.05, max_depth = 8, gamma = 0)

In [ ]:
import xgboost as xgb
xgb.plot_importance(bst)

In [ ]:
###ngram features
train.get_ngram_features(n = 1, remove_stopwords=False, suffix = '_stops')
train.get_ngram_features(n = 1, remove_stopwords=True,)
train.get_ngram_features(n = 2, remove_stopwords=False, suffix = '_stops')
train.get_ngram_features(n = 2, remove_stopwords=True,)
# train.get_ngram_features(n = 3, remove_stopwords=False, suffix = '_stops')
# train.get_ngram_features(n = 3, remove_stopwords=True,)
# train.get_ngram_features(n = 4, remove_stopwords=False, suffix = '_stops')
# train.get_ngram_features(n = 4, remove_stopwords=True,)
# train.get_ngram_features(n = 5, remove_stopwords=False, suffix = '_stops')
# train.get_ngram_features(n = 5, remove_stopwords=True,)

In [ ]:
bst = model.train_xgb(train.features,train.target, num_rounds = 400, eta = 0.1, max_depth = 4, gamma = 0)

In [ ]:
bst = model.train_xgb(train.features,train.target, num_rounds = 400, eta = 0.1, max_depth = 4, gamma = 0)
xgb.plot_importance(bst)

In [ ]:
train.get_fuzzy_features(remove_stopwords = False, suffix = '_stops')
train.get_fuzzy_features(remove_stopwords = True)

In [ ]:
bst = model.train_xgb(train.features,train.target, num_rounds = 400, eta = 0.1, max_depth = 4, gamma = 0)
xgb.plot_importance(bst)

In [ ]:
train.get_vector_features(vectorizer = 'QuoraQuestions', n_principle_directions = 10, suffix = 'QQ')
train.get_vector_features(vectorizer = 'GoogleNews', n_principle_directions = 10, suffix = 'GG')

In [ ]:
bst = model.train_xgb(train.features,train.target, num_rounds = 400, eta = 0.1, max_depth = 4, gamma = 0)
xgb.plot_importance(bst)

In [ ]:
from src.model import reweight_training_set
X,y = reweight_training_set(train.features, train.target)

In [ ]:
reload(model)
bst = model.train_xgb(X,y, num_rounds = 400, eta = 0.1, max_depth = 8, gamma = 0)
xgb.plot_importance(bst)

In [ ]:
dval = xgb.DMatrix(X_val, label = y_val, missing = np.NAN)
prop = bst.predict(dval)

In [ ]:
features = train.features
features['target'] = train.target
plt.figure()
feature = 'frac_common_1grams'
features.loc[features.target == 0, feature].plot.hist(normed = True, bins = 30, ax = plt.gca(), alpha = 0.5)
features.loc[features.target == 1, feature].plot.hist(normed = True, bins = 30, ax = plt.gca(),alpha = 0.5)





In [ ]:
sbn.lmplot(
    'frac_common_1grams', 
    'Tsort', 
    data = features, 
    hue = 'target', 
    fit_reg=False,
    scatter_kws={"marker": "D",
                 "s": 10,
                 'alpha' : 0.2
                },
    
)

In [ ]:
pd.DataFrame(train.target.values, columns = ['is_duplicate']).to_csv('data/target.csv')

In [ ]:
xgb.plot_importance?